# Text-to-Query Demo with Medha and MongoDB (MQL)

This notebook demonstrates Medha with **MongoDB Query Language** (MQL), including
`find()`, `aggregate()`, and `countDocuments()`. This proves Medha works with
document-oriented query languages beyond SQL and graph queries.

**No live MongoDB instance is required.** Medha caches query strings without
executing them. The demo shows cache behavior across all four tiers:

- **Tier 0** — L1 in-memory cache (microseconds)
- **Tier 1** — Template matching with parameter extraction
- **Tier 2** — Exact vector match
- **Tier 3** — Semantic similarity match

**Requirements:** `pip install medha[fastembed]`

## Cell 1: Setup & Imports

In [ ]:
import time

from medha import Medha, Settings
from medha.embeddings.fastembed_adapter import FastEmbedAdapter
from medha.types import QueryTemplate

## Cell 2: Define Sample MongoDB Queries

These are string representations of MongoDB queries and aggregation pipelines.
No live MongoDB connection is needed — Medha caches the query strings.

The queries cover three MQL patterns:
- **`find()`** — filter and retrieve documents
- **`aggregate()`** — grouping and computed fields
- **`countDocuments()`** — counting with optional filters

In [ ]:
pairs = [
    (
        "Find all active users",
        'db.users.find({"status": "active"})',
    ),
    (
        "How many orders are there?",
        "db.orders.countDocuments({})",
    ),
    (
        "Get products cheaper than 50 dollars",
        'db.products.find({"price": {"$lt": 50}})',
    ),
    (
        "Show the top 3 highest paid employees",
        'db.employees.find().sort({"salary": -1}).limit(3)',
    ),
    (
        "Average salary by department",
        'db.employees.aggregate([{"$group": {"_id": "$department", "avg_salary": {"$avg": "$salary"}}}])',
    ),
    (
        "Count users per city",
        'db.users.aggregate([{"$group": {"_id": "$city", "count": {"$sum": 1}}}])',
    ),
]

print(f"Prepared {len(pairs)} question-MQL pairs")
for question, query in pairs:
    print(f"  Q: {question}")
    print(f"  M: {query}\n")

## Cell 3: Initialize Medha

We use:
- **FastEmbedAdapter** — local embeddings, no API key needed
- **memory mode** — Qdrant runs in-process, no external server required
- **query_language="generic"** — MQL is not a built-in language label, so we use generic

In [ ]:
embedder = FastEmbedAdapter()
settings = Settings(qdrant_mode="memory", query_language="generic")

medha = Medha(collection_name="mongodb_demo", embedder=embedder, settings=settings)
await medha.start()
print("Medha initialized (collection='mongodb_demo', mode='memory', language='generic')")

## Cell 4: Store Question-MQL Pairs

We store all 6 pairs. Since there is no live MongoDB, we omit `response_summary` —
the cached queries are self-descriptive MQL strings.

In [ ]:
for question, query in pairs:
    await medha.store(question, query)
    print(f"Stored: {question!r}")

## Cell 5: Tier 2 — Exact Vector Match

Searching with the **exact same question** that was stored yields a high-confidence
exact vector match. The returned query is MQL — Medha is query-language agnostic.

In [ ]:
start = time.perf_counter()
hit = await medha.search("How many orders are there?")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # EXACT_MATCH or L1_CACHE
print(f"Query:    {hit.generated_query}")  # db.orders.countDocuments({})
print(f"Score:    {hit.confidence:.4f}")
print(f"Time:     {elapsed:.2f}ms")

## Cell 6: Tier 3 — Semantic Similarity Match

We search with a **rephrased question** that was never stored. Medha recognizes
the semantic similarity and returns the matching MQL query from the cache.

In [ ]:
start = time.perf_counter()
hit = await medha.search("List all users that are currently active")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # SEMANTIC_MATCH
print(f"Query:    {hit.generated_query}")  # db.users.find({"status": "active"})
print(f"Score:    {hit.confidence:.4f}")
print(f"Time:     {elapsed:.2f}ms")

## Cell 7: Tier 1 — Template Matching with Parameter Extraction

We load two MongoDB templates:
- **`find_by_field`** — 3 parameters: collection, field, value
- **`count_collection`** — 1 parameter: collection name

Each template has `parameter_patterns` with regex patterns for extraction.
We test with "Find users where status is pending" — a combination that was
**not** in any stored pair ("pending" is a new value).

In [ ]:
templates = [
    QueryTemplate(
        intent="find_by_field",
        template_text="Find {collection} where {field} is {value}",
        query_template='db.{collection}.find({{"{field}": "{value}"}})',
        parameters=["collection", "field", "value"],
        parameter_patterns={
            "collection": r"\b(users|products|orders|employees)\b",
            "field": r"\b(status|category|department|city)\b",
            "value": r"\b(active|pending|completed|Engineering|Marketing|Sales)\b",
        },
    ),
    QueryTemplate(
        intent="count_collection",
        template_text="Count all {collection}",
        query_template="db.{collection}.countDocuments({})",
        parameters=["collection"],
        parameter_patterns={
            "collection": r"\b(users|products|orders|employees)\b",
        },
    ),
]

await medha.load_templates(templates)
print(f"Loaded {len(templates)} template(s)\n")

# Test find_by_field template — "pending" is a new value not in stored pairs
start = time.perf_counter()
hit = await medha.search("Find users where status is pending")
elapsed = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit.strategy}")        # TEMPLATE_MATCH
print(f"Query:    {hit.generated_query}")  # db.users.find({"status": "pending"})
print(f"Time:     {elapsed:.2f}ms\n")

# Test count_collection template
start = time.perf_counter()
hit2 = await medha.search("Count all products")
elapsed2 = (time.perf_counter() - start) * 1000

print(f"Strategy: {hit2.strategy}")        # TEMPLATE_MATCH
print(f"Query:    {hit2.generated_query}")  # db.products.countDocuments({})
print(f"Time:     {elapsed2:.2f}ms")

## Cell 8: Tier 0 — L1 In-Memory Cache

The L1 cache stores recent search results in memory. The first call goes through the
full waterfall. The second call for the **same question** returns instantly from the
L1 cache — typically >100x faster.

In [ ]:
# First call — goes through the vector backend
start = time.perf_counter()
hit1 = await medha.search("Average salary by department")
t1 = (time.perf_counter() - start) * 1000

# Second call — served from L1 cache
start = time.perf_counter()
hit2 = await medha.search("Average salary by department")
t2 = (time.perf_counter() - start) * 1000

print(f"First call:  {t1:.2f}ms ({hit1.strategy})")
print(f"Second call: {t2:.2f}ms ({hit2.strategy})")
if t2 > 0:
    print(f"Speedup:     {t1/t2:.0f}x")

## Cell 9: Stats & Cleanup

In [ ]:
print("Cache Statistics:")
print(medha.stats)

await medha.close()
print("\nCleaned up: Medha closed.")